In [5]:
import PyQt6.QtCore
import os
os.environ["QT_API"] = "pyqt6"
import matplotlib.pyplot as plt

# Use qt backend for matplotlab to use interactive mne plots
%matplotlib qt

import mne 
import analysis.processing
import pandas as pd
import csv 
import os
from config import Config
configObj = Config()
from mne_connectivity import spectral_connectivity_time
import numpy as np
configss = configObj.getConfigSnapshot()
from tqdm import tqdm
import tools.helpers
from scipy import stats

mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [11]:
[el for el in list(range(1,32)) if el not in [14, 5, 13, 16, 17, 20] ]

[1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 15,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [14]:
listsss =  [el for el in list(range(1,32)) if el not in [14, 5, 13, 16, 17, 20, 31] ]
df = None
for pnum in tqdm(listsss):

    partipant_data_path =  str(pnum) +'.csv'
    path_qa = os.path.join(configss['root'], configss['data_qa'] , partipant_data_path ) 

    if df is None:
        df =  pd.read_csv(path_qa)
    else:
        df = pd.concat([df, pd.read_csv(path_qa)], axis=0)   

# Calculate means and standard errors for each BlockType
means = df.groupby('BlockType')['Correct'].mean()
counts = df.groupby('BlockType')['Correct'].count()
stds = df.groupby('BlockType')['Correct'].std()
stderr = stds / np.sqrt(counts)

# 95% Confidence Intervals (CI = mean ± 1.96 * standard error)
confidence_intervals = 1.96 * stderr

# Perform a t-test to calculate p-value between block types
block_type_d = df[df['BlockType'] == 'D']['Correct']
block_type_nd = df[df['BlockType'] == 'ND']['Correct']
t_stat, p_value = stats.ttest_ind(block_type_d, block_type_nd)

# Plotting with error bars
plt.figure(figsize=(8,6))
plt.bar(means.index, means, yerr=confidence_intervals, capsize=10)

# Adding titles, labels, and annotations
plt.title(f'Question Accuracy for Each Block Type (p-value = {p_value:.4f})')
plt.xlabel('Block Type')
plt.ylabel('Accuracy (Proportion of Correct Answers)')
plt.xticks(rotation=0)

# Display the plot
plt.show()

100%|██████████| 24/24 [00:00<00:00, 568.65it/s]


In [4]:


path_qa = os.path.join(configss['root'], configss['data_qa'] , partipant_data_path ) 
path_tones = os.path.join(configss['root'], configss['data_tones'] , partipant_data_path ) 
path_completion = os.path.join(configss['root'], configss['data_completion'] , partipant_data_path ) 
path_reading_data = os.path.join(configss['root'], configss['data_reading_data'] , partipant_data_path ) 

KeyError: 'data_qa'

In [ ]:


# Calculate mean and standard error for each block type
accuracy_stats = df_combined.groupby('BlockType')['Correct'].agg(['mean', 'count'])
accuracy_stats['stderr'] = np.sqrt((accuracy_stats['mean'] * (1 - accuracy_stats['mean'])) / accuracy_stats['count'])

# Calculate the 95% confidence interval using the standard error
confidence_interval = stats.norm.ppf(0.975) * accuracy_stats['stderr']

# Plotting the accuracy with error bars
plt.figure(figsize=(8,6))
plt.bar(accuracy_stats.index, accuracy_stats['mean'], yerr=confidence_interval, capsize=5)

# Adding titles and labels
plt.title('Question Accuracy for Each Block Type (with 95% CI)')
plt.xlabel('Block Type')
plt.ylabel('Accuracy (Proportion of Correct Answers)')
plt.xticks(rotation=0)

# Show plot
plt.show()